In [1]:
# Run this cell to start.
import numpy as np
import pandas as pd
import os

 
import seaborn as sns
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

#### Run this notebook first, then Visualisation.ipynb, then Analysis.ipynb.

## Our Data Sources and How To Access Them:

### Generating accurate European population data per year for per Capita comparison:
- **Country Total Population per Year**
    - https://data.worldbank.org/indicator/SP.POP.TOTL
    - Download as an xlsx file.
    - Save as 'WorldBankWorldPop2019.xls'
- **England & Wales Population Data by Year**
    - https://statswales.gov.wales/Catalogue/Population-and-Migration/Population/Estimates/Local-Authority/populationestimates-by-localauthority-year
    - Save as 'uk_allpopulation_byregion.csv'
    - Missing 2017 and 2018 so used google analytics of source Eurostat
    - https://www.google.com/search?q=wales+population&sxsrf=ALeKk0092HQ2WF07rymTKr-ElWoAyo_vqA%3A1621786290686&ei=sn6qYL21KYCPjLsPmOSG0AQ&oq=wales+population&gs_lcp=Cgdnd3Mtd2l6EAMyBAgjECcyBwgAELEDEEMyBQgAELEDMgIIADIECAAQQzIECAAQQzICCAAyAggAMgIIADICCAA6CggAEIcCELEDEBQ6BwgAEIcCEBQ6BwgjEOoCECc6BQgAEJECOgQILhBDOg0ILhCxAxDHARCjAhBDOggIABCxAxCDAToKCAAQsQMQgwEQQzoHCC4QsQMQQzoICAAQsQMQkQI6BQgAEMkDOgIILlDeE1i1MmDGM2gDcAJ4AIABdYgBlguSAQQxOS4xmAEAoAEBqgEHZ3dzLXdperABCsABAQ&sclient=gws-wiz&ved=0ahUKEwi91ofemODwAhWAB2MBHRiyAUoQ4dUDCBE&uact=5
    - Google analytics from search engine allowed us to manually find and add in 2017 and 2018 population for England and Wales from source 'Eurostat' for populations from 2017 and 2018.

### Generating accurate per Capita comparison metrics data for the aims of drug decriminalization:
- **EMCDDA Overdose Deaths Total per Country in Europe**
    - https://www.emcdda.europa.eu/data/stats2020/drd
    - Open 'National trends' then 'Number of Deaths' then 'Total'. Download as an xlsx file.
    - Save as 'overdoseDeaths_Total.xlsx'
- **EMCDDA Number of Use Offences Total per Country in Europe**
    - https://www.emcdda.europa.eu/data/stats2020/dlo
    - Open 'Offences by type' and then download 'Use Offences' as xlsx file.
    - Save as 'n_UseOffences.xlsx'
- **EMCDDA Number of Offenders Total per Country in Europe**
    - https://www.emcdda.europa.eu/data/stats2020/dlo
    - Open 'Number of offences' and then download 'Offenders' as xlsx file.
    - Save as 'n_Offenders.xlsx'
- **Drug Decriminalisation Date and Status**
    - https://www.talkingdrugs.org/drug-decriminalisation
    - Manually manipulated data and tables based on interactive infographic's information.
    - Source adequately references its information in the interactive infographic.
    
### Generating a Drug Use and Possession Law Severity Scoring Index AND a Reliablity Single-blind Test of the Index (built for multiple participants)
- **EMCDDA Drug Laws at a Glance**
    - https://www.emcdda.europa.eu/publications/topic-overviews/content/drug-law-penalties-at-a-glance_en
    - If the download data doesn't work, scroll to the bottom of the page and copy and paste the txt information into notepad. Then save as a csv.
    - Save as 'European_Drug_Laws_Raw.csv'.
- **Questionnaire: Drug Law Leniency Scoring**
    - Public file folder (public access link): https://drive.google.com/file/d/1Xx1-3m6l22TPRgpeS35tObiFyHjasOoj/view?usp=sharing
    - Download author scores as :'Drug_Laws_Leniency_Scoring_Author_Scoring.xlsx'
    - Download participant one's form as: 'Drug_Laws_Leniency_Scoring_Participant_One.xlsx'
    - Download participant two's form as: 'Drug_Laws_Leniency_Scoring_Participant_Two.xlsx'
    



## Simulating Data

### Generating accurate European population data per year for per Capita comparison:
Through getting country populations by year to create a per capita function for our values in later dataframes.

In [2]:
# Load in the World Bank's Population data per country per year.
WorldPopData = pd.read_excel('WorldBankWorldPop2019.xls', header = 3)

# Transpose the data so countries are Columns
WorldPopDataTransposed = WorldPopData.T
WorldPopDataTransposed = WorldPopDataTransposed.reset_index()
# Make the country columns the headers.
WorldPopDataTransposed.columns = WorldPopDataTransposed.iloc[0]
# Drop unnecessary rows
WorldPopDataTransposed = WorldPopDataTransposed.drop([0,1,2,3])
# Change the 'Country' column to 'Date' and sort by ascending to have the earliest dates first. Note this is not possible if not all dates are integers and must be manipulated otherwise.
WorldPopDataTransposed = WorldPopDataTransposed.sort_values(by='Country Name').rename(columns={'Country Name': 'Date'})
WorldPopDataTransposed = WorldPopDataTransposed.drop([64])
# Used print(df.column.values) to check names of columns (countries) to see if they match up to what we're using or are out of date.
WorldPopDataTransposed.rename(columns={'Czech Republic' : 'Czechia', 'Slovak Republic' : 'Slovakia'},inplace=True)
# Ensuring no strings objects in Date column
WorldPopDataTransposed['Date'] = WorldPopDataTransposed.loc[: ,'Date'].astype('int')

WorldPopDataClean = WorldPopDataTransposed.copy()

# Remember that not all tables call certain countries the same things, especially Czechia (once Czech Republic) and Slovakia (once Slovak Republic). In our tables Certain countries have * by them which needs removing and investigating too. United Kingdom also missing Scotland for some tables.

# A list of the countries that submit data to the EMCDDA.
european_list = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus' , 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Turkey', 'United Kingdom']

# This list variant doesn't include Scotland or NI data in the United Kingdom (England and Wales only)
european_list_variant = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus' , 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Turkey', 'UK (England & Wales)']

# Loading in the UK population per year by Country.
uk_pop_data = pd.read_csv('uk_allpopulation_byregion.csv')
# Selecting only England and Wales region data and storing it in a series. Covers enough dates for our EMCDDA analysis.
uk_pop_data['Eng_Wales'] = uk_pop_data.iloc[:,2] + uk_pop_data.iloc[:,3]
england_wales_pop = uk_pop_data['Eng_Wales']

# Selecting only countries relevant to our analysis using EMCDDA data here.
EuropePopData =  WorldPopDataClean[european_list].copy()
# Fixing column order
EuropePopData.insert(0, "Date", WorldPopDataClean.loc[:, 'Date'].copy())
# Inserting 'England and Wales' as a separate 'country' to call on for analysis.
EuropePopData['UK (England & Wales)'] = england_wales_pop
EuropePopData['UK (England & Wales)'] = EuropePopData['UK (England & Wales)'].shift(periods=32)
EuropePopData = EuropePopData.set_axis(EuropePopData['Date'], axis=0).copy()
# This is stuff we're trying to do to make our function work
EuropePopData = EuropePopData.drop('Date',axis=1)
# Populations from Eurostat added manually as the full dataset downloaded previously was missing 2017 and 2018.
england_2017 = 55620000
wales_2017 = 3119000
england_2018 = 55980000
wales_2018 = 3127000
EuropePopData.loc[2017,'UK (England & Wales)'] = (england_2017 + wales_2017)
EuropePopData.loc[2018,'UK (England & Wales)'] = (england_2018 + wales_2018)
EuropePopData.head()


,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,...,Poland,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom,UK (England & Wales)
Date,,,,,,,,,,,,,,,,,,,,,
1960,7047539.0,9153489.0,7867374.0,4140181.0,572930.0,9602006.0,4579603.0,1211537.0,4429634.0,46621669.0,...,29637450.0,8857716.0,18406905.0,4068095.0,1584720.0,30455000.0,7484656.0,27472345.0,52400000.0,NaN
1961,7086299.0,9183948.0,7943118.0,4167292.0,576395.0,9586651.0,4611687.0,1225077.0,4461005.0,47240543.0,...,29964000.0,8929316.0,18555250.0,4191667.0,1594131.0,30739250.0,7519998.0,28146910.0,52800000.0,NaN
1962,7129864.0,9220578.0,8012946.0,4196712.0,577691.0,9624660.0,4647727.0,1241623.0,4491443.0,47904877.0,...,30308500.0,8993985.0,18676550.0,4238188.0,1603649.0,31023366.0,7561588.0,28832827.0,53250000.0,NaN
1963,7175811.0,9289770.0,8078145.0,4225675.0,577913.0,9670685.0,4684483.0,1258857.0,4523309.0,48582611.0,...,30712000.0,9030355.0,18797850.0,4282017.0,1616971.0,31296651.0,7604328.0,29531365.0,53650000.0,NaN
1964,7223801.0,9378113.0,8144340.0,4252876.0,578625.0,9727804.0,4722072.0,1277086.0,4548543.0,49230595.0,...,31139450.0,9035365.0,18919126.0,4327341.0,1632114.0,31609195.0,7661354.0,30244261.0,54000000.0,NaN



### 2. Generating accurate per Capita comparison metrics data for the aims of drug decriminalization:
- **EMCDDA Overdose Deaths per 1,000,0000 per Country in Europe between 1985 and 2018**
This metric gives us a direct insight into health of addicts and users.

In [3]:
# A function that looks between two tables where country name is in a column. It is to be applied on melted (long data format) EMCDDA table to match country and date to value - and divide that value by the relevant country's total population for that year. Then it is multiplied by 1,000,000 to find the per capita value of that event per 1,000,000 people. 
def per_capita_1000000(row):
    country = row.loc['Country']
    date = row.loc['Date']
    CountryTotPop = EuropePopData.loc[date, country]
    return (row.loc['value'] / CountryTotPop) * 1000000

# This allows comparison between countries' values, where before we would only have the total numbers per country.

In [4]:
# Loading in the EMCDDA data for total number of overdose deaths per year for data-submitting European countries.
overdoseDeaths_Total_Europe = pd.read_excel('overdoseDeaths_Total.xlsx', header = 3)

overdoseDeaths_Total_Europe.head()

,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1993,1992,1991,1990,1989,1988,1987,1986,1985,Unnamed: 35
0,Austria,184.0,154.0,165.0,153.0,122.0,138.0,161.0,201.0,187.0,...,156.0,129.0,71.0,43.0,40.0,NaN,NaN,NaN,NaN,NaN
1,Belgium,NaN,NaN,NaN,NaN,61.0,74.0,72.0,94.0,106.0,...,123.0,64.0,63.0,50.0,26.0,33.0,17.0,NaN,NaN,NaN
2,Bulgaria,24.0,18.0,22.0,17.0,15.0,21.0,24.0,25.0,41.0,...,9.0,8.0,10.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Croatia,85.0,65.0,56.0,54.0,59.0,48.0,49.0,59.0,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cyprus,12.0,16.0,6.0,9.0,6.0,3.0,5.0,8.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# PREPARING THE TABLE
# The table is transposed to get the countries in the columns and the date in the index.
overdoseDeaths_Total_Europe_Transposed = overdoseDeaths_Total_Europe.T
# The index is replaced and renamed, with the columns then being tidied.
overdoseDeaths_Total_Europe_Transposed = overdoseDeaths_Total_Europe_Transposed.reset_index()
overdoseDeaths_Total_Europe_Transposed.columns = overdoseDeaths_Total_Europe_Transposed.iloc[0]
overdoseDeaths_Total_Europe_Transposed = overdoseDeaths_Total_Europe_Transposed.iloc[ :, 0:32]
# We rename the first column to stay on track after the transposition of the table. Country to 'Date'
overdoseDeaths_Total_Europe_Transposed = overdoseDeaths_Total_Europe_Transposed.rename(columns={'Country': 'Date'})
overdoseDeaths_Total_Europe_Clean = overdoseDeaths_Total_Europe_Transposed.drop([0, 35]).copy()
# Some dates were considered 'string' type data, so they had to be converted to integers.
overdoseDeaths_Total_Europe_Clean['Date'] = overdoseDeaths_Total_Europe_Clean['Date'].astype('int')
# Sorted by date just so we can see what's going on.
overdoseDeaths_Total_Europe_Clean = overdoseDeaths_Total_Europe_Clean.sort_values(by='Date')
# We faced some problems with visualisation later on due to the full name being too long, so we shortened it.
overdoseDeaths_Total_Europe_Clean = overdoseDeaths_Total_Europe_Clean.rename(columns={'United Kingdom (England & Wales)': 'UK (England & Wales)'})



# APPLYING THE FUNCTION
# The dataset is melted into a long-table form so we can apply the per_capita_1000000 function to each row.
# As it is the (England and Wales) only variant, as Scotland and NI did not provide data, we select value_vars with the variant list we created.
overdoseDeaths_Total_Europe_melt = pd.melt(overdoseDeaths_Total_Europe_Clean, id_vars=['Date'], value_vars=european_list_variant)
overdoseDeaths_Total_Europe_melt = overdoseDeaths_Total_Europe_melt.rename(columns={0: 'Country'})
# We drop NAN values so the function will work.
overdoseDeaths_Total_Europe_melt = overdoseDeaths_Total_Europe_melt.dropna(subset=['value'])
# Here we make sure that all values are indeed integers.
overdoseDeaths_Total_Europe_melt['value'] = overdoseDeaths_Total_Europe_melt['value'].astype('int')
# The function is applied to the melted table row by row and the output is copied into a new column in the table.
overdoseDeaths_Total_Europe_melt['Overdoses per 1000000 People'] = overdoseDeaths_Total_Europe_melt.apply(per_capita_1000000, axis='columns').copy()
# We drop the totals 'value' columns as we have a per capita column to work on instead.
overdosedeaths_europe = overdoseDeaths_Total_Europe_melt.drop(columns= 'value').copy()

# FINALISING THE FORMAT AND EXPORTING
# Using the .pivot function to return the table into a tidy format (from long table format) with Date as the index.
overdosedeaths_europe_unmelted = overdosedeaths_europe.pivot(index='Date', columns='Country')
overdosedeaths_europe_unmelted.columns = overdosedeaths_europe_unmelted.columns.droplevel(0)
overdosedeaths_per_capita = overdosedeaths_europe_unmelted.copy()
# Writing it to CSV to call in other ipynbs in the folder.
overdosedeaths_per_capita.to_csv('overdosedeaths_per_capita.csv')


overdosedeaths_per_capita

Country,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,...,Norway,Poland,Portugal,Romania,Slovakia,Slovenia,Spain,Sweden,Turkey,UK (England & Wales)
Date,,,,,,,,,,,,,,,,,,,,,
1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.312253,...,10.836804,NaN,NaN,NaN,NaN,NaN,3.639246,NaN,NaN,NaN
1986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.211453,...,10.558258,NaN,NaN,NaN,NaN,NaN,3.913476,NaN,NaN,NaN
1987,NaN,1.722350,NaN,NaN,NaN,NaN,NaN,NaN,3.244039,2.443167,...,10.270116,NaN,NaN,NaN,NaN,NaN,5.842086,8.335512,NaN,NaN
1988,NaN,3.332773,NaN,NaN,NaN,NaN,NaN,NaN,2.223803,2.793824,...,11.402812,NaN,NaN,NaN,NaN,NaN,8.435022,6.519300,NaN,NaN
1989,5.249642,2.616300,NaN,NaN,NaN,NaN,NaN,NaN,4.633014,3.900573,...,10.646097,NaN,NaN,NaN,NaN,NaN,10.971531,6.593693,NaN,NaN
1990,5.600526,5.016364,2.752834,NaN,NaN,NaN,23.925590,NaN,8.222314,6.010059,...,16.503700,4.067091,NaN,NaN,NaN,NaN,11.423478,7.243977,NaN,NaN
1991,9.155512,6.297175,1.158431,NaN,NaN,NaN,32.788170,NaN,6.781365,7.018525,...,20.648882,5.569182,NaN,NaN,NaN,NaN,14.191723,7.194766,NaN,NaN
1992,16.452594,6.371229,0.936750,NaN,NaN,NaN,32.099811,NaN,5.355026,8.479009,...,24.262779,5.187200,NaN,NaN,NaN,NaN,13.688245,7.614154,NaN,NaN
1993,19.732765,12.196966,1.062284,NaN,NaN,NaN,36.040356,NaN,5.131801,7.681015,...,21.567763,5.486019,NaN,NaN,NaN,NaN,10.695795,9.519920,NaN,42.364942


- **EMCDDA Use Offences per 1,000,0000 per Country in Europe between 2007 and 2018**

This metric gives us an insight into drug addict and user suffering at the hands of law enforcement, as well as potential for administrative burden on law enforcement agencies.

In [6]:
n_UseOffences_Europe = pd.read_excel('n_UseOffences.xlsx', header = 3)
n_UseOffences_Europe


/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,Unnamed: 13
0,Austria,NaN,NaN,NaN,NaN,28067.0,25348.0,20554.0,21884.0,20306.0,19735.0,17121.0,20492.0,NaN
1,Belgium,40688.0,38573.0,35847.0,33782.0,35320.0,30312.0,26500.0,26447.0,23458.0,26655.0,25655.0,23923.0,NaN
2,Bulgaria,NaN,NaN,NaN,NaN,5432.0,5249.0,4906.0,3716.0,4077.0,2767.0,2269.0,2654.0,NaN
3,Croatia,8903.0,8874.0,8722.0,6709.0,7292.0,5546.0,5189.0,5269.0,5132.0,4690.0,5004.0,5513.0,NaN
4,Cyprus,1008.0,811.0,737.0,802.0,917.0,816.0,854.0,792.0,669.0,560.0,596.0,693.0,NaN
5,Czechia,NaN,NaN,881.0,856.0,2836.0,2600.0,1911.0,1544.0,1364.0,1171.0,861.0,NaN,NaN
6,Denmark,22243.0,20672.0,16704.0,NaN,21412.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,3020.0,4289.0,4352.0,3633.0,2862.0,3619.0,3842.0,2999.0,2278.0,3352.0,6528.0,5991.0,NaN
8,Finland,19286.0,17315.0,15715.0,15170.0,13681.0,12738.0,11308.0,12121.0,12185.0,11268.0,9933.0,10406.0,NaN
9,France,161000.0,164113.0,159702.0,166390.0,176652.0,170337.0,161325.0,157024.0,135447.0,137594.0,147909.0,112923.0,NaN


In [7]:
# PREPARING THE TABLE
# The table is transposed to get the countries in the columns and the date in the index.
n_UseOffences_Europe_transposed = n_UseOffences_Europe.T
# The index is replaced and renamed, with the columns then being tidied.
n_UseOffences_Europe_transposed = n_UseOffences_Europe_transposed.reset_index()
n_UseOffences_Europe_transposed.columns = n_UseOffences_Europe_transposed.iloc[0]
n_UseOffences_Europe_transposed = n_UseOffences_Europe_transposed.iloc[0:29, 0:31]
# Some countries are named inconsistently. The notations are not relevant enough for us to classify them differently.
n_UseOffences_Europe_transposed = n_UseOffences_Europe_transposed.rename(columns={'Romania-B' : 'Romania', 'Sweden *' : 'Sweden', 'Italy *' : 'Italy'})
# Correctly naming the date column.
n_UseOffences_Europe_Clean = n_UseOffences_Europe_transposed.rename(columns={'Country': 'Date'}).copy()
# We create a workaround to have all data values as integers.
arr_use = n_UseOffences_Europe_Clean['Date'][1:13] # index 0 (string) was causing the problem, so do not index it.
arr2_use = arr_use.astype('int')
n_UseOffences_Europe_Clean['Date'] = arr2_use
# Sort for convenience
n_UseOffences_Europe_Clean = n_UseOffences_Europe_Clean.sort_values(by='Date')
# Drop the empty/useless columns
n_UseOffences_Europe_Clean = n_UseOffences_Europe_Clean.drop([0, 13]).copy()

# APPLYING THE FUNCTION
# Creating a melted long form table with Date as the index. Value vars is the normal european list of country names.
n_UseOffences_Europe_melt = pd.melt(n_UseOffences_Europe_Clean, id_vars=['Date'], value_vars=european_list)
# Renaming the 0 column to Country.
n_UseOffences_Europe_melt = n_UseOffences_Europe_melt.rename(columns={0: 'Country'})
# Dropping all NA value rows so the function doesn't stall.
n_UseOffences_Europe_melt = n_UseOffences_Europe_melt.dropna(subset=['value'])


# Making sure all values are integers - because of the workaround used earlier. Double making sure!
n_UseOffences_Europe_melt['value'] = n_UseOffences_Europe_melt['value'].astype('int')
# Applying the per capita 1000000 function to each row in the long form table, with returns in a new row. 
n_UseOffences_Europe_melt['Drug Use Offences per 1,000,000 People'] = n_UseOffences_Europe_melt.apply(per_capita_1000000, axis='columns').copy()
# Dropping the total number value, as we now have the per capita value. 
n_UseOffences_Europe = n_UseOffences_Europe_melt.drop(columns= 'value')


# FINALISING THE FORMAT AND EXPORTING
# Pivoting the table back into tidy form with Date as the index and countries as columns.
n_UseOffences_Europe_unmelted = n_UseOffences_Europe.pivot(index='Date', columns='Country')
# Correcting the header.
n_UseOffences_Europe_unmelted.columns = n_UseOffences_Europe_unmelted.columns.droplevel(0)

# Countries with no data points between 2015-2018 4 year period are excluded after looking at the table.
n_UseOffences_Europe_unmelted= n_UseOffences_Europe_unmelted.drop(columns = ['Austria','Bulgaria', 'Romania'])
n_UseOffences_per_capita = n_UseOffences_Europe_unmelted.copy()
# Writing it to CSV to call in other ipynbs in the folder.
n_UseOffences_per_capita.to_csv('n_UseOffences_per_capita.csv')
n_UseOffences_per_capita

Country,Belgium,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,Germany,Greece,...,Malta,Norway,Poland,Portugal,Slovakia,Slovenia,Spain,Sweden,Turkey,United Kingdom
Date,,,,,,,,,,,,,,,,,,,,,
2007.0,2251.428141,1279.053932,651.491521,NaN,NaN,4468.627860,1967.583839,1763.974680,2084.642799,NaN,...,1175.244146,4325.193936,1075.692487,639.668313,84.471057,1495.449730,5606.387876,3123.711480,NaN,1528.037124
2008.0,2395.430876,1161.100354,551.054354,82.911210,NaN,4882.244277,1869.424826,2297.616105,2062.913164,NaN,...,1140.752212,4024.779100,1029.041809,619.709255,96.853957,1741.439735,6595.014600,3403.387791,NaN,1615.690910
2009.0,2468.857248,1089.385092,509.979665,112.122479,NaN,2511.773940,2110.558581,2126.414684,2071.846401,NaN,...,1120.062452,4120.962755,1208.940028,714.309573,115.104580,1580.648625,8028.609744,11498.502718,NaN,1566.214547
2010.0,2152.981951,1194.758984,601.287780,130.222132,NaN,1710.884545,2271.900110,2082.918541,2028.444942,NaN,...,687.562122,4660.631115,1231.429006,691.850072,116.666679,1809.543475,7220.489592,8438.146384,1006.899389,1566.189152
2011.0,2395.938347,1230.895884,704.104520,147.102425,NaN,2259.237524,2249.515243,2403.081106,2121.420568,NaN,...,622.195316,4465.093291,1315.862240,653.370665,115.960628,1797.994294,8378.827606,8563.570321,792.496440,1592.220910
2012.0,2385.897384,1215.917862,752.392414,181.813252,NaN,2904.673485,2088.670220,2456.982475,2155.240612,NaN,...,604.721590,4676.030417,1329.737065,815.323556,115.763450,1752.902911,7928.667477,8694.689378,960.898887,1461.172462
2013.0,2716.273365,1303.196733,713.370272,247.282931,NaN,2745.833261,2341.986684,2580.914981,2353.296252,NaN,...,755.927102,4982.653240,NaN,834.728293,99.198414,1946.646356,8530.944146,8817.985207,1071.573012,1392.864579
2014.0,3151.023320,1720.465016,795.810064,269.444798,3794.116214,2177.179176,2504.983968,2663.949534,2587.151545,1548.968075,...,936.583839,4802.391638,664.899932,870.968753,104.269533,1973.346007,8571.739237,8722.570185,782.668749,1281.099939
2015.0,2996.399921,1596.011423,690.792732,81.167761,NaN,2761.882824,2768.485113,2500.290316,2617.932087,1606.708066,...,806.645501,NaN,714.281728,1002.116609,76.698979,NaN,8415.209684,8622.552934,700.018002,1123.836751


- **EMCDDA Number of Drug-related Offenders per 1,000,0000 per Country in Europe between 2007 and 2018**

This metric serves to see if decriminalization of drug use reduces administrative burden of law enforcement agencies, without only looking at use offences.

In [8]:
n_Offenders_Europe = pd.read_excel('n_Offenders.xlsx', header = 3)
n_Offenders_Europe.head(10)

/usr/local/lib/python3.8/dist-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Country,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,Unnamed: 13
0,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Belgium,54344.0,51172.0,49014.0,46224.0,47906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bulgaria,1989.0,2028.0,3759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Croatia,9993.0,9951.0,10078.0,NaN,8555.0,NaN,5545.0,5715.0,5423.0,5019.0,5225.0,5679.0,NaN
4,Cyprus,1305.0,1094.0,1030.0,1111.0,1268.0,1165.0,1243.0,1052.0,973.0,831.0,930.0,1028.0,NaN
5,Czechia,11216.0,10208.0,4499.0,3816.0,3925.0,6887.0,4350.0,3928.0,4550.0,3092.0,2322.0,2031.0,NaN
6,Denmark,29542.0,17781.0,NaN,NaN,27094.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,847.0,881.0,613.0,NaN
8,Finland,27587.0,25211.0,23081.0,22015.0,20610.0,21625.0,19178.0,20133.0,19855.0,18620.0,16777.0,16491.0,NaN
9,France,199024.0,199024.0,192120.0,199667.0,213976.0,207285.0,197580.0,197572.0,157341.0,159412.0,175691.0,134620.0,NaN


In [9]:
# PREPARING THE TABLE
# The table is transposed to get the countries in the columns and the date in the index.
n_Offenders_Europe_transposed = n_Offenders_Europe.T
# The index is replaced and renamed, with the columns then being tidied.
n_Offenders_Europe_transposed = n_Offenders_Europe_transposed.reset_index()
n_Offenders_Europe_transposed.columns = n_Offenders_Europe_transposed.iloc[0]
n_Offenders_Europe_transposed = n_Offenders_Europe_transposed.iloc[1:13,0:31]
# Correctly naming the date column.
n_Offenders_Europe_transposed = n_Offenders_Europe_transposed.rename(columns={'Country':'Date'})
# Making sure all our values are integers.
n_Offenders_Europe_transposed['Date'] = n_Offenders_Europe_transposed.loc[:,'Date'].astype('int')
n_Offenders_Europe_transposed = n_Offenders_Europe_transposed.sort_values(by='Date')
# Some countries are named inconsistently. The notations are not relevant enough for us to classify them differently.
n_Offenders_Europe_transposed = n_Offenders_Europe_transposed.rename(columns={'Latvia *' : 'Latvia', 'Italy *' : 'Italy'})
n_Offenders_Europe_clean = n_Offenders_Europe_transposed.copy()

# APPLYING THE FUNCTION
# Creating a melted long form table with Date as the index. Value vars is the normal european list of country names.
n_Offenders_Europe_melt = pd.melt(n_Offenders_Europe_clean, id_vars=['Date'], value_vars= european_list)

# Renaming the 0 column to Country.
n_Offenders_Europe_melt = n_Offenders_Europe_melt.rename(columns={0: 'Country'})
# Dropping all NA value rows so the function doesn't fail
n_Offenders_Europe_melt = n_Offenders_Europe_melt.dropna(subset=['value'])
# Making sure all values are integers.
n_Offenders_Europe_melt['value'] = n_Offenders_Europe_melt['value'].astype('int')
# Applying the per capita 1000000 function to each row in the long form table, with returns in a new row. 
n_Offenders_Europe_melt['Offenders per 1,000,000 People'] = n_Offenders_Europe_melt.apply(per_capita_1000000, axis='columns').copy()
# Dropping the total number value, as we now have the per capita value.
n_Offenders_Europe_melt = n_Offenders_Europe_melt.drop(columns= 'value')


# FINALISING THE FORMAT AND EXPORTING
# Pivoting the table back into tidy form with Date as the index and countries as columns.
offenders_europe_unmelted = n_Offenders_Europe_melt.pivot(index='Date', columns='Country')
# Correcting the header.
offenders_europe_unmelted.columns = offenders_europe_unmelted.columns.droplevel(0)
offenders_europe_unmelted


# # Countries with no data points between 2015-2018 4 year period are excluded after looking at the table.
offenders_europe_unmelted= offenders_europe_unmelted.drop(columns = ['Sweden', 'Estonia'])
offenders_per_capita = offenders_europe_unmelted.copy()
# Writing it to CSV to call in other ipynbs in the folder.
offenders_per_capita.to_csv('offenders_per_capita.csv')



- **Drug Decriminalisation Date and Status**

This table serves as the basis for most boolean selection of specific data points, containing categorical information on countries.

In [10]:
# Creating a function that allows us to plot individual countries with consistent line colours between different tables and different graphs (rather than relying on seaborn, which would change the colours etcetera).

# https://seaborn.pydata.org/generated/seaborn.color_palette.html
# In order to visualise countries with consistent colours (that seaborn doesn't offer as a package) I started the mission of finding out how seaborn actually selects random colours for as many variables as it's presented in the hope I could do this and input the colour selections in a fixed table - and plot these colours consistently. After doing a lot of research I found the easiest way would be to select the palettes I wanted, and then copy the hex codes printed on the notebook.

colours = pd.DataFrame()
pal1 = sns.color_palette('colorblind', 10)
pal2 = sns.color_palette('dark', 10)
pal3 = sns.color_palette('husl', 9)

# using print(pal3.as_hex()) etc and then copy and pasting them into lists.
# This is a workaround and there are probably better ways to do it for bulk, but this was quick enough for what I wanted to do for one graph. It also only required slightly editing some previously written functions.

coloursinput1 = ['#0173b2', '#de8f05', '#029e73', '#d55e00', '#cc78bc', '#ca9161', '#fbafe4', '#949494', '#ece133', '#56b4e9']
colourinput2 = ['#001c7f', '#b1400d', '#12711c', '#8c0800', '#591e71', '#592f0d', '#a23582', '#3c3c3c', '#b8850a', '#006374']
colourinput3 = ['#f77189', '#d58c32', '#a4a031', '#50b131', '#34ae91', '#37abb5', '#3ba3ec', '#bb83f4', '#f564d4']


def classify_country_colour(row):
    if row.loc['Country'] == 'Austria':
        return  "#0173b2"
    if row.loc['Country'] == 'Belgium':
        return  "#de8f05"
    if row.loc['Country'] == 'Bulgaria':
        return  "#029e73"
    if row.loc['Country'] == 'Croatia':
        return  "#d55e00"
    if row.loc['Country'] == 'Cyprus':
        return  "#cc78bc"
    if row.loc['Country'] == 'Czechia':
        return  "#ca9161"
    if row.loc['Country'] == 'Denmark':
        return  "#fbafe4"
    if row.loc['Country'] == 'Estonia':
        return  "#949494"
    if row.loc['Country'] == 'Finland':
        return  "#ece133"
    if row.loc['Country'] == 'France':
        return  "#56b4e9"
    if row.loc['Country'] == 'Germany':
        return  "#001c7f"
    if row.loc['Country'] == 'Greece':
        return  "#b1400d"
    if row.loc['Country'] == 'Hungary':
        return  "#12711c"
    if row.loc['Country'] == 'Ireland':
        return  "#8c0800"
    if row.loc['Country'] == 'Austria':
        return  "#591e71"
    if row.loc['Country'] == 'Italy':
        return  "#592f0d"
    if row.loc['Country'] == 'Latvia':
        return  "#a23582"
    if row.loc['Country'] == 'Lithuania':
        return  "#3c3c3c"
    if row.loc['Country'] == 'Luxembourg':
        return  "#b8850a"
    if row.loc['Country'] == 'Malta':
        return  "#006374"
    if row.loc['Country'] == 'Netherlands':
        return  "#f77189"
    if row.loc['Country'] == 'Norway':
        return  "#d58c32"
    if row.loc['Country'] == 'Poland':
        return  "#a4a031"
    if row.loc['Country'] == 'Portugal':
        return  "#50b131"
    if row.loc['Country'] == 'Romania':
        return  "#34ae91"
    if row.loc['Country'] == 'Slovakia':
        return  "#37abb5"
    if row.loc['Country'] == 'Slovenia':
        return  "#3ba3ec"
    if row.loc['Country'] == 'Spain':
        return  "#bb83f4"
    if row.loc['Country'] == 'Sweden':
        return  "#f564d4"
    if row.loc['Country'] == 'Turkey':
        return  "#006400"
    if row.loc['Country'] == 'United Kingdom' or 'UK (England & Wales)':
        return  "#000000"

In [11]:
# CREATING THE CATEGORICAL DATAFRAME FROM SCRATCH WITH NON-TABULAR FORMATTED ONLINE RESOURCES (see sources at top)
# Creating lists with the same element numbers, for information on each country that has decriminalised drugs in the European Union. (e.g Switzerland is left out because we have no data for it)
decriminalised_european_names = np.array(['Czechia', 'Croatia', 'Estonia', 'Germany','Italy', 'Netherlands','Poland', 'Portugal', 'Spain'])
decriminalised_european_date = np.array([1990, 2013, 2002, 1992, 1990, 1973, 2013, 2001, 1983])
decriminalised_european_decrim_form = ['De jure', 'De jure', 'De jure', 'De jure', 'De jure' , 'De facto' , 'De jure', 'De jure', 'De jure']
decriminalised_european_evidence = np.array(['Amendment no. 175/1990 Coll. to the Penal Act no. 140/1961 Coll.', 'Amendments to the Criminal Code 1997.', 'Act on Narcotic Drugs and Psychotropic Substances and Precursors.', 'German Federal Narcotics Act (1992 amendment)', 'Decree of the President of the Republic No 309 of 9 October 1990.', 'De Facto', 'Amendment to Act on Counteracting Drug Addiction 2011 (article 62a).', 'Law 30/2000.', 'Ley Organic 8/1983.'])

# An array of the criminalised european union names we have data for is added.
criminalised_european_names = np.array(['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Denmark', 'Finland', 'France', 'Greece', 'Hungary', 'Ireland', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Norway', 'Romania', 'Slovakia', 'Slovenia', 'Sweden', 'Turkey', 'United Kingdom', 'UK (England & Wales)'])

# A table containing the decriminalised country's categorical data is created.
decriminalised_european_table = pd.DataFrame()
decriminalised_european_table['Country'] = decriminalised_european_names
decriminalised_european_table['Decriminalised'] = 'Yes'
decriminalised_european_table['Date of Decriminalisation'] = decriminalised_european_date
decriminalised_european_table['Form'] = decriminalised_european_decrim_form
decriminalised_european_table['Law'] = decriminalised_european_evidence
europe_decriminalised_status = decriminalised_european_table.copy()

# A table containing the criminalised country's categorical data is created.
criminalised_european_table = pd.DataFrame()
criminalised_european_table['Country'] = criminalised_european_names
criminalised_european_table['Decriminalised'] = 'No'
criminalised_european_table['Date of Decriminalisation'] = float("NaN")
criminalised_european_table['Form'] = 'None'
criminalised_european_table['Law'] = 'None'


# We concatenate the two tables. (sticking them together with the columns that are same not being duplicated)
concat_frames = [decriminalised_european_table, criminalised_european_table]
concat_table = pd.concat(concat_frames).reset_index()
europe_criminalisation_status = concat_table.iloc[: , 1:]

# Applying the function to each row and return its values in the column. Each country will have its individual hex code colour.
europe_criminalisation_status_with_colour = europe_criminalisation_status.copy()
europe_criminalisation_status_with_colour['Colour'] = europe_criminalisation_status.apply(classify_country_colour, axis='columns')
# Decriminalised and criminalised countries are assigned a different colour for plotting purposes.
europe_criminalisation_status_with_colour['StatusColour'] = 'Salmon'
europe_criminalisation_status_with_colour.loc[0:8, 'StatusColour'] = 'Cornflowerblue'

# Setting the index as country for easy future loc selection.
europe_criminalisation_status_with_colour = europe_criminalisation_status_with_colour.set_index(['Country'])
# Saving as a CSV for use in other pages.
europe_criminalisation_status_with_colour.to_csv('europe_criminalisation_status_with_colour.csv')
europe_criminalisation_status_with_colour



# It is built this way so we can call upon country names in the index through loc while plotting to retrieve specific colours for differentiation of status per line/plot.
# Also our functions tend to follow the same format, of applying to rows one at a time so keeping it standardised like this gives us less headaches.

,Decriminalised,Date of Decriminalisation,Form,Law,Colour,StatusColour
Country,,,,,,
Czechia,Yes,1990.0,De jure,Amendment no. 175/1990 Coll. to the Penal Act ...,#ca9161,Cornflowerblue
Croatia,Yes,2013.0,De jure,Amendments to the Criminal Code 1997.,#d55e00,Cornflowerblue
Estonia,Yes,2002.0,De jure,Act on Narcotic Drugs and Psychotropic Substan...,#949494,Cornflowerblue
Germany,Yes,1992.0,De jure,German Federal Narcotics Act (1992 amendment),#001c7f,Cornflowerblue
Italy,Yes,1990.0,De jure,Decree of the President of the Republic No 309...,#592f0d,Cornflowerblue
Netherlands,Yes,1973.0,De facto,De Facto,#f77189,Cornflowerblue
Poland,Yes,2013.0,De jure,Amendment to Act on Counteracting Drug Addicti...,#a4a031,Cornflowerblue
Portugal,Yes,2001.0,De jure,Law 30/2000.,#50b131,Cornflowerblue
Spain,Yes,1983.0,De jure,Ley Organic 8/1983.,#bb83f4,Cornflowerblue


### Generating a Drug Use and Possession Law Severity Scoring Index AND a Reliability Single-blind Test of the Index (built for multiple participants)

With the goal being to see whether it is formal decriminalisation that affects the metrics presented, or whether a countries individual legal approach regardless of decriminalisation makes a difference, we built our own metric to judge 'strictness' of laws regarding use and possession of drugs. We look at the laws summarised, generate a key that we feel encompasses every country's legal approach, assign them a score from least to most strict for that category of law.

Cons of our strictness metric were that specific categories with more different approaches had increased 'weight' in the 'total strictness scores' calculated at the end. This was unavoidable unless we wanted to put a lot more work into working on the metric after analysis.

To increase reliability of our crafted strictness metric, we managed to find two volunteers who were willing to re-score countries per law single-blind meaning they were not influenced by country names/bias when scoring leniency of law. Means could then be taken and compared between author and participant responses etc to judge reliability of the key and scoring.



- **Instructions**

In [12]:
# Excel was chosen as it is a common table-visualising and form-filling software that our participants likely have access to.
# In order to help participants fill the form correctly, we created instructions to add to an excel sheet.
Instructions = pd.DataFrame()
Instructions['Steps'] = np.arange(0, 5)
Instructions['Instructions'] = ['In the second sheet you will see rows of anonymous countries, and columns of laws for different aspects of drug law per country' , 'Please refer to sheet 3 for a key and try to score each law based on the key', 'If you are unsure or feel like it straddles categories, please score if how you think it best fits.', 'A reccommended way of doing this is opening the different sheets on different screens or devices.', 'Then please save your scoring file and send it to upload it to this google drive folder https://drive.google.com/drive/folders/1Sd9Y1Vg_F9eXA5AsfBjZgB2y5579FaBH?usp=sharing']

Instructions

,Steps,Instructions
0,0,In the second sheet you will see rows of anony...
1,1,Please refer to sheet 3 for a key and try to s...
2,2,If you are unsure or feel like it straddles ca...
3,3,A reccommended way of doing this is opening th...
4,4,Then please save your scoring file and send it...


- **Country-anonymous form with drug laws**

In [13]:
# We built a table from the EMCDDA 'laws at a glance' data. This can be found here.
# https://www.emcdda.europa.eu/publications/topic-overviews/content/drug-law-penalties-at-a-glance_en

European_Drug_Laws_Raw = pd.read_csv('European_Drug_Laws_Raw.csv')

# As decriminalisation mainly focuses on treating addiction as a health issues, but still often regards supply as a criminal one, we wanted to look at lenience of laws affecting addicts and users. We thus eliminated supply laws as a factor.
European_Drug_Laws_Raw = European_Drug_Laws_Raw.iloc[:, :-8]
# Recividism for use offences was eliminated as it was rarely codified for most countries. Use by quantity was also rarely codified, probably because it becomes a supply offence governed by different laws after a certain amount
European_Drug_Laws_Raw = European_Drug_Laws_Raw.drop(columns = ['Penalty varies by quantity? [USE]', 'Penalty varies for recidivism? [USE]'])
# We then insert blank columns for our participants to fill out.
European_Drug_Laws_Raw.insert(loc=2, column='Punishment for Use', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=4, column='Alternative Punishments for Use', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=6, column='Penalty Variance by Drug Type Used', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=8, column='Penalty Variance by Addiction Presence for Use', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=10, column='Punishment for Possession', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=12, column='Alternative Punishments for Possession', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=14, column='Penalty Variance by Drug Possessed', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw.insert(loc=16, column='Penalty Variance by Quantity', value=['' for i in range(European_Drug_Laws_Raw.shape[0])])
European_Drug_Laws_Raw['Penalty Variance by Addiction Presence for Possession'] = ""
num_samples = len(European_Drug_Laws_Raw['COUNTRY'])

# Here we create the column used to anonymise the country names by replacing them with numbers.
European_Drug_Laws_Raw.insert(loc=1, column='CODE', value=np.arange(0, num_samples))

European_Drug_Laws_Blank_NonAnon = European_Drug_Laws_Raw.copy()
# Using iloc we select all but the country name column, and shuffle our rows with the sample method.
European_Drug_Laws_Blank_Anon = European_Drug_Laws_Blank_NonAnon.iloc[:, 1:].sample(frac = 1)
# This is the sheet particpants fill out, with the key to aid them. It is 'single blinded' so they do not know which country is which.
# In excel they do not have access to the 'index' like we see here in pandas.
European_Drug_Laws_Blank_Anon

,CODE,What is the punishment for the offence? [USE],Punishment for Use,What are the alternatives to punishment for the offence? [USE],Alternative Punishments for Use,Penalty varies by drug? [USE],Penalty Variance by Drug Type Used,Penalty (response) varies for addiction? [USE],Penalty Variance by Addiction Presence for Use,What is the punishment for the offence? [POSSESSION],Punishment for Possession,What are the alternatives to punishment for the offence? [POSSESSION],Alternative Punishments for Possession,Penalty varies by drug? [POSSESSION],Penalty Variance by Drug Possessed,Penalty varies by quantity? [POSSESSION],Penalty Variance by Quantity,Penalty (response) varies for addiction? [POSSESSION],Penalty Variance by Addiction Presence for Possession
14,14,Use of drugs is not mentioned as an offence.,,"There is no alternative to punishment for use,...",,Use of drugs per se not regulated by the law.,,Use of drugs per se not regulated by the law.,,Possession of drugs for personal use is a non-...,,A socio-rehabilitation and therapeutic program...,,Class of drug determines duration of administr...,,"Penalty doesn't vary by quantity, but quantity...",,Sanctions do not vary for addiction. DPR 309/9...,
6,6,Use of drugs is not mentioned as an offence.,,"There is no alternative to punishment for use,...",,Use of drugs per se not regulated by the law.,,Use of drugs per se not regulated by the law.,,Possession of drugs is punished by up to 2 yea...,,No alternatives to punishment are specified fo...,,Penalty ranges do not vary according to the ty...,,The law does not directly differentiate the pu...,,Possession of drugs punished by a fine or impr...,
29,29,Only use of prepared opium is explicitly prohi...,,Drug Intervention Programmes as alternatives t...,,"Use of drugs is not regulated by the law, exce...",,enalty does not vary by addiction factor. Misu...,,Possession of drugs is a criminal offence. Pun...,,Drug intervention programmes as alternatives t...,,Punishment is linked to the class of drugs inv...,,Punishment is linked to the class of drugs inv...,,Penalty does not vary by addiction factor. Mis...,
4,4,Use of controlled drugs is a criminal offence ...,,The Law providing treatment alternatives to pu...,,Penalty for use offences does not vary by drug...,,Penalty does not vary by addiction factor. Nar...,,Possession of controlled drugs for personal us...,,The Law providing treatment alternatives to pu...,,Possession of controlled drugs for personal us...,,"Penalty range does not vary by quantity, but t...",,"Penalty does not vary by addiction factor, but...",
24,24,Use of drugs is not mentioned as an offence.,,"There is no alternative to punishment for use,...",,Use of drugs is not mentioned as an offence.,,Use of drugs is not mentioned as an offence.,,Possession of drugs is punished by up to 3 yea...,,Court may order â€œprotective treatmentâ€ in ...,,Penalty does not vary for different drugs. Pen...,,Possession of drugs for personal use is punish...,,Penalty range is not influenced by addiction f...,
1,1,Use of drugs is not mentioned as an offence.,,"There is no alternative to punishment for use,...",,Use of drugs per se not regulated by the law.,,Use of drugs per se not regulated by the law.,,Possession of drugs is punishable by 3 months ...,,Cases of problem drug users may be settled wit...,,Possession of cannabis for personal use within...,,Limit for cannabis possession for personal use...,,Penalty range does not vary for addiction. How...,
3,3,Use of drugs in public is punishable with a fi...,,There are no alternatives to the administrativ...,,Drug use per se not regulated by the law. Admi...,,Administrative sanctions do not vary by addict...,,\r\n\t\t\tPossession of drugs for personal use...,,As well as the fine or imprisonment up to 90 d...,,Possession of drugs for personal use is an adm...,,Possession of drugs for personal use is an adm...,,Penalty (response) varies for addiction. If an...,
2,2,Only use of high-risk an

- **Key**

In [14]:
# Here we manually code a key table for another sheet in the excel file.
# The keys and key values were chosen to encompass what we judged to be all discrete different approaches in each law variable for the European countries, ranked on the strictness of those particular approaches. It is important to note that not all law approaches perfectly fit into a category - so some are 'judgement calls'. We will test how effective our key is in getting a reliable view of leniency and strictness for law approaches by seeing if helpful participants put the same or similar laws in each category as we do.

Key = pd.DataFrame()
Key['Punishment [USE]'] = ['No punishment' ,
'Fine/Administrative punishment only',
'Fine or imprisonment',
'Imprisonment',
'Fine and imprisonment',
'> 10 years imprisonment']
Key['Punishment [USE] Class'] = [1, 2, 3, 4 ,5 , 6]

Key['Alternative options [USE]'] = ['No alternative (not an offence)',
'Voluntary Alternative treatment',
'Compulsory treatment (alternative) Court decision',
'Compulsory treatment (additive)',
'No alternative (an offence)', ""]
Key['Alternative options [USE] Class'] = [1,2,3,4,5, ""]

Key['Varies by drug [USE]'] = ['Use of drugs isnt an offence',
'Only an offence for high risk substances',
'Penalty does not vary by drug', "", "" , ""]
Key['Varies by drug [USE] Class'] = [1,2,3,"","", ""]

Key['Varies by addiction [USE]'] = ['Use of drugs isnt an offence',
'No penalty for offenders with drug addiction',
'Addicted offenders may be referred to alternative treatment.',
'Penalty does not vary by addiction', "", ""]
Key['Varies by addiction [USE] Class'] = [1,2,3,4,"", ""]


Key['Punishment [POSSESSION]'] = ['No punishment'
,'Fine/Administrative punishment only'
,'Fine or imprisonment'
,'Imprisonment'
,'Fine and imprisonment'
,'> 10 years imprisonment]']
Key['Punishment [POSSESSION] Class'] = [1, 2, 3, 4 ,5 , 6]
 
Key['Alternative options [POSSESSION]'] =['No alternative (not an offence)',
'Voluntary Alternative treatment',
'Compulsory treatment (alternative) Court decision',
'Compulsory treatment (additive)',
'No alternative (an offence)', ""]
Key['Alternative options [POSSESSION] Class'] = [1,2,3,4,5, ""]


Key['Varies by drug [POSSESSION]'] = ['Possession of drugs isnt a criminal offence (personal amount)', 
'Penalty varies depending on drug',
'Penalty does not vary by drug (offence)' , "" ,"", ""]
Key['Varies by drug [POSSESSION] Class'] = [1,2,3,"","", ""]


Key['Varies by quantity [POSSESSION]'] = ['Possession of drugs is not a criminal offence',
'Possession of personal amounts is not an offence',
'Possession of personal amounts is an offence, but is punished less harshly',
'Penalty does not vary by quantity', "", ""]
Key['Varies by quantity [POSSESSION] Class'] = [1,2,3,4,"", ""]


Key['Varies by addiction [POSSESSION]'] = ['Possession of drugs isnt an offence',
'No penalty for offenders with drug addiction',
'Addicted offenders may be referred to alternative treatment.',
'Penalty does not vary by addiction', "", ""]
Key['Varies by addiction [POSSESSION] Class'] = [1,2,3,4,"", ""]




# We write the dataframes as different sheets for excel to give to our participants.
with pd.ExcelWriter('Drug_Laws_Leniency_Scoring_Anonymous.xlsx') as writer:
    Instructions.to_excel(writer, sheet_name='Instructions')
    European_Drug_Laws_Blank_Anon.to_excel(writer, sheet_name='European_Drug_Laws_Blank_Anon')
    Key.to_excel(writer, sheet_name='Key')
    
# It is also available in an excel-friendly formatted version at 'Scoring_Blank.xlsx' at this public google drive folder location https://drive.google.com/drive/folders/1Sd9Y1Vg_F9eXA5AsfBjZgB2y5579FaBH?usp=sharing
# The data and submissions from participants who helped in single blinding the trial also uploaded to this drive.

Key

,Punishment [USE],Punishment [USE] Class,Alternative options [USE],Alternative options [USE] Class,Varies by drug [USE],Varies by drug [USE] Class,Varies by addiction [USE],Varies by addiction [USE] Class,Punishment [POSSESSION],Punishment [POSSESSION] Class,Alternative options [POSSESSION],Alternative options [POSSESSION] Class,Varies by drug [POSSESSION],Varies by drug [POSSESSION] Class,Varies by quantity [POSSESSION],Varies by quantity [POSSESSION] Class,Varies by addiction [POSSESSION],Varies by addiction [POSSESSION] Class
0,No punishment,1,No alternative (not an offence),1,Use of drugs isnt an offence,1,Use of drugs isnt an offence,1,No punishment,1,No alternative (not an offence),1,Possession of drugs isnt a criminal offence (p...,1,Possession of drugs is not a criminal offence,1,Possession of drugs isnt an offence,1
1,Fine/Administrative punishment only,2,Voluntary Alternative treatment,2,Only an offence for high risk substances,2,No penalty for offenders with drug addiction,2,Fine/Administrative punishment only,2,Voluntary Alternative treatment,2,Penalty varies depending on drug,2,Possession of personal amounts is not an offence,2,No penalty for offenders with drug addiction,2
2,Fine or imprisonment,3,Compulsory treatment (alternative) Court decision,3,Penalty does not vary by drug,3,Addicted offenders may be referred to alternat...,3,Fine or imprisonment,3,Compulsory treatment (alternative) Court decision,3,Penalty does not vary by drug (offence),3,"Possession of personal amounts is an offence, ...",3,Addicted offenders may be referred to alternat...,3
3,Imprisonment,4,Compulsory treatment (additive),4,,,Penalty does not vary by addiction,4,Imprisonment,4,Compulsory treatment (additive),4,,,Penalty does not vary by quantity,4,Penalty does not vary by addiction,4
4,Fine and imprisonment,5,No alternative (an offence),5,,,,,Fine and imprisonment,5,No alternative (an offence),5,,,,,,
5,> 10 years imprisonment,6,,,,,,,> 10 years imprisonment],6,,,,,,,,


### Calculating Total Strictness Scores from Form Responses and Analysis/Visualisation Preparation

In [15]:


# Creating separate lists of column names related to use law scoring, posession law scoring, and those two together incase we want to split up analysis after looking at totals only.
column_names_use= ['Punishment for Use', 'Alternative Punishments for Use', 'Penalty Variance by Drug Type Used', 'Penalty Variance by Addiction Presence for Use']
column_names_possession = ['Punishment for Possession', 'Alternative Punishments for Possession', 'Penalty Variance by Drug Possessed', 'Penalty Variance by Quantity', 'Penalty Variance by Addiction Presence for Possession']
column_names_use_and_possession = np.append(column_names_use, column_names_possession)
# For selection of scores for the author response
column_names_use_and_possession_by_country = np.append('Country' , column_names_use_and_possession)
# For selection of scores for the participant response
column_names_use_and_possession_by_code = np.append('CODE' , column_names_use_and_possession)



# Only loading in the columns we want in the first place. The submissions were posted into a public google drive you can access manually with steps listed at the top.
Drug_Laws_Leniency_Scoring_Author = pd.read_excel('Drug_Laws_Leniency_Scoring_Author_Scoring.xlsx').loc[:, column_names_use_and_possession_by_country].reset_index().copy()
Drug_Laws_Leniency_Scoring_One = pd.read_excel('Drug_Laws_Leniency_Scoring_Participant_One.xlsx', sheet_name = 1).loc[:, column_names_use_and_possession_by_code].sort_values('CODE',ascending = True).reset_index().copy()
Drug_Laws_Leniency_Scoring_Two = pd.read_excel('Drug_Laws_Leniency_Scoring_Participant_Two.xlsx', sheet_name = 1).loc[:, column_names_use_and_possession_by_code].sort_values('CODE',ascending = True).reset_index().copy()


# This section of code is for 'decodifying' and standardising all response columns. The author template was used as a master to decode the country-codified forms.
Drug_Laws_Leniency_Scoring_Author = Drug_Laws_Leniency_Scoring_Author.set_index('Country')
Drug_Laws_Leniency_Scoring_Author = Drug_Laws_Leniency_Scoring_Author.drop(columns = 'index')

Drug_Laws_Leniency_Scoring_One['CODE'] = Drug_Laws_Leniency_Scoring_Author.index.values
Drug_Laws_Leniency_Scoring_One = Drug_Laws_Leniency_Scoring_One.rename(columns = {'CODE' : 'Country'})
Drug_Laws_Leniency_Scoring_One.index = Drug_Laws_Leniency_Scoring_Author.index
# Selecting only the relevant columns (less error prone than dropping them for this table)
Drug_Laws_Leniency_Scoring_One = Drug_Laws_Leniency_Scoring_One.iloc[:, 2:]


Drug_Laws_Leniency_Scoring_Two['CODE'] = Drug_Laws_Leniency_Scoring_Author.index.values
Drug_Laws_Leniency_Scoring_Two = Drug_Laws_Leniency_Scoring_Two.rename(columns = {'CODE' : 'Country'})
Drug_Laws_Leniency_Scoring_Two.index = Drug_Laws_Leniency_Scoring_Author.index
# Selecting only the relevant columns (less error prone than dropping them for this table)
Drug_Laws_Leniency_Scoring_Two = Drug_Laws_Leniency_Scoring_Two.iloc[:, 2:]
Drug_Laws_Leniency_Scoring_Two.head()

,Punishment for Use,Alternative Punishments for Use,Penalty Variance by Drug Type Used,Penalty Variance by Addiction Presence for Use,Punishment for Possession,Alternative Punishments for Possession,Penalty Variance by Drug Possessed,Penalty Variance by Quantity,Penalty Variance by Addiction Presence for Possession
Country,,,,,,,,,
Austria,1,1,1,1,3,2,3,4,4
Belgium,1,1,1,1,3,3,2,2,3
Bulgaria,2,1,2,4,3,4,2,3,4
Croatia,2,5,3,3,3,4,3,4,3
Cyprus,6,5,3,4,4,1,2,4,4


In [16]:
# Working out totals for use, posession and use and posession strictness law scores.
Drug_Laws_Leniency_Scoring_Author_wTotals = Drug_Laws_Leniency_Scoring_Author.copy()
Drug_Laws_Leniency_Scoring_Author_wTotals['Total Strictness Score for Use'] = Drug_Laws_Leniency_Scoring_Author_wTotals[column_names_use].sum(axis=1)
Drug_Laws_Leniency_Scoring_Author_wTotals['Total Strictness Score for Possession'] = Drug_Laws_Leniency_Scoring_Author_wTotals[column_names_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_Author_wTotals['Total Strictness Score for Use and Possession'] = Drug_Laws_Leniency_Scoring_Author_wTotals[column_names_use_and_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_Author_TotOnly = Drug_Laws_Leniency_Scoring_Author_wTotals.copy()
Drug_Laws_Leniency_Scoring_Author_TotOnly = Drug_Laws_Leniency_Scoring_Author_TotOnly.iloc[:, -3:]


Drug_Laws_Leniency_Scoring_One_wTotals = Drug_Laws_Leniency_Scoring_One.copy()
Drug_Laws_Leniency_Scoring_One_wTotals['Total Strictness Score for Use'] = Drug_Laws_Leniency_Scoring_One_wTotals[column_names_use].sum(axis=1)
Drug_Laws_Leniency_Scoring_One_wTotals['Total Strictness Score for Possession'] = Drug_Laws_Leniency_Scoring_One_wTotals[column_names_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_One_wTotals['Total Strictness Score for Use and Possession'] = Drug_Laws_Leniency_Scoring_One_wTotals[column_names_use_and_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_One_TotOnly = Drug_Laws_Leniency_Scoring_One_wTotals.copy()
Drug_Laws_Leniency_Scoring_One_TotOnly = Drug_Laws_Leniency_Scoring_One_TotOnly.iloc[:, -3:]


Drug_Laws_Leniency_Scoring_Two_wTotals = Drug_Laws_Leniency_Scoring_Two.copy()
Drug_Laws_Leniency_Scoring_Two_wTotals['Total Strictness Score for Use'] = Drug_Laws_Leniency_Scoring_Two_wTotals[column_names_use].sum(axis=1)
Drug_Laws_Leniency_Scoring_Two_wTotals['Total Strictness Score for Possession'] = Drug_Laws_Leniency_Scoring_Two_wTotals[column_names_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_Two_wTotals['Total Strictness Score for Use and Possession'] = Drug_Laws_Leniency_Scoring_Two_wTotals[column_names_use_and_possession].sum(axis=1)
Drug_Laws_Leniency_Scoring_Two_TotOnly = Drug_Laws_Leniency_Scoring_Two_wTotals.copy()
Drug_Laws_Leniency_Scoring_Two_TotOnly = Drug_Laws_Leniency_Scoring_Two_TotOnly.iloc[:, -3:]


# As the tables are in the same format and order we can simply add them together and divide by 3 to get mean values for each cell! This is the mean values table to be compared to how we scored it.
Drug_Laws_Leniency_Scoring_Mean_TotOnly = (Drug_Laws_Leniency_Scoring_Author_TotOnly + Drug_Laws_Leniency_Scoring_Two_TotOnly + Drug_Laws_Leniency_Scoring_One_TotOnly) /3

Drug_Laws_Leniency_Scoring_Mean_TotOnly.head()


,Total Strictness Score for Use,Total Strictness Score for Possession,Total Strictness Score for Use and Possession
Country,,,
Austria,4.000000,15.000000,19.000000
Belgium,4.000000,12.333333,16.333333
Bulgaria,11.666667,15.666667,27.333333
Croatia,10.666667,15.000000,25.666667
Cyprus,18.000000,17.333333,35.333333


In [17]:
# Here we realised that we could define the strictness parameters by fixed values to capture the whole range of scored points e.g 10-20 is lenient, 20-30 is moderate etc. However, this is rather arbitrary and more dependent on 'score weights' set by the key rather than the spread of the score. 

# The second idea was to split it into equal proportions of 33.33*% for each category (lenient, moderate, strict). We decided against this incase countries with very similar/same were assigned to a different category purely based on order. Also it may have grouped strict countries with very high scores with more moderate countries with higher strictness scores - that despite this had scores much closer to other moderate countries than groupings of strict countries.

# We thus went with creating individual sized 'bins' based on the total range presented in the total strictness scores column. With our functions, the parameters for countries are dynamic dependent on the range of each response (e.g mean or author)



# First we write our nested function to create 'parameters' to define equal size bin ranges dependent upon minimum and maximum strictness score values.

def bin_values(df, column):
    binsize = np.max(df[column]) - np.min(df[column])
    start, bin1, bin2 = np.arange(np.min(df[column]), np.max(df[column]), binsize/3)
    bin3 = bin2 + (binsize/3)
    return start, bin1, bin2, bin3

# Wrote the functions out individually because we want to .apply to EACH ROW and multiple arguments will make that more complicated. If we had a lot more responses it would have been worth working this out fully into one function.

def bin_categorise_author_tot_use_pos(row):
    start, bin1, bin2, bin3 = bin_values(Drug_Laws_Leniency_Scoring_Author_TotOnly, 'Total Strictness Score for Use and Possession')
    if row.loc['Total Strictness Score for Use and Possession'] >= start and row.loc['Total Strictness Score for Use and Possession'] <= bin1:
        return 'Lenient'
    if row.loc['Total Strictness Score for Use and Possession'] > bin1 and row.loc['Total Strictness Score for Use and Possession'] <= bin2:
        return 'Moderate'
    if row.loc['Total Strictness Score for Use and Possession'] >= bin2:
        return 'Strict'

def bin_categorise_mean_tot_use_pos(row):
    start, bin1, bin2, bin3 = bin_values(Drug_Laws_Leniency_Scoring_Author_TotOnly, 'Total Strictness Score for Use and Possession')
    if row.loc['Total Strictness Score for Use and Possession'] >= start and row.loc['Total Strictness Score for Use and Possession'] <= bin1:
        return 'Lenient'
    if row.loc['Total Strictness Score for Use and Possession'] > bin1 and row.loc['Total Strictness Score for Use and Possession'] <= bin2:
        return 'Moderate'
    if row.loc['Total Strictness Score for Use and Possession'] >= bin2:
        return 'Strict'

    

    
# A function to be applied to each row, to assign a colour to each country dependent upon strictness. For plotting purposes.
def colour_by_category(row):
    if row.loc['Category'] == 'Lenient':
        return 'green'
    if row.loc['Category'] == 'Moderate':
        return 'orange'
    if row.loc['Category'] == 'Strict':
        return 'red'
# A function to be applied to each row, to assign a value to each country dependent upon strictness. For data analysis purposes - specifically seeing if there is any difference between how the average respondent to the key survey responded, versus our author response values to the key we built.
def value_by_category(row):
    if row.loc['Category'] == 'Lenient':
        return 1
    if row.loc['Category'] == 'Moderate':
        return 2
    if row.loc['Category'] == 'Strict':
        return 3

    
# Here we apply the functions and save the tables for exporting for analysis/plotting.
Drug_Laws_Leniency_Scoring_Author_TotOnly['Category'] = Drug_Laws_Leniency_Scoring_Author_TotOnly.apply(bin_categorise_author_tot_use_pos, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Author_TotOnly['ColourCategory'] = Drug_Laws_Leniency_Scoring_Author_TotOnly.apply(colour_by_category, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Author_TotOnly['CategoryValue'] = Drug_Laws_Leniency_Scoring_Author_TotOnly.apply(value_by_category, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Author_TotOnly.to_csv('Drug_Laws_Leniency_Scoring_Author_TotOnly.csv')

Drug_Laws_Leniency_Scoring_Mean_TotOnly['Category'] = Drug_Laws_Leniency_Scoring_Mean_TotOnly.apply(bin_categorise_mean_tot_use_pos, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Mean_TotOnly['ColourCategory'] = Drug_Laws_Leniency_Scoring_Mean_TotOnly.apply(colour_by_category, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Mean_TotOnly['CategoryValue'] = Drug_Laws_Leniency_Scoring_Mean_TotOnly.apply(value_by_category, axis='columns').copy()
Drug_Laws_Leniency_Scoring_Mean_TotOnly.to_csv('Drug_Laws_Leniency_Scoring_Mean_TotOnly.csv')

Drug_Laws_Leniency_Scoring_Mean_TotOnly.head()

,Total Strictness Score for Use,Total Strictness Score for Possession,Total Strictness Score for Use and Possession,Category,ColourCategory,CategoryValue
Country,,,,,,
Austria,4.000000,15.000000,19.000000,Lenient,green,1
Belgium,4.000000,12.333333,16.333333,Lenient,green,1
Bulgaria,11.666667,15.666667,27.333333,Moderate,orange,2
Croatia,10.666667,15.000000,25.666667,Moderate,orange,2
Cyprus,18.000000,17.333333,35.333333,Strict,red,3


In [18]:
# This is a quick workaround to transform the table again, as it was worked on on different notebooks so some lines of code call country as an index, and some as a column. 
# This quick patch was to reduce headaches.

reset_index_author_tot_only = Drug_Laws_Leniency_Scoring_Author_TotOnly.reset_index()
reset_index_mean_tot_only = Drug_Laws_Leniency_Scoring_Mean_TotOnly.reset_index()

In [19]:
# Here we create tables of only category values to show in analysis the differences between how specific countries were scored with our author scoring and mean scoring. Low mis-categorising would mean more reliable key.

Drug_Laws_Leniency_Scoring_Author_Categories = pd.DataFrame()
Drug_Laws_Leniency_Scoring_Author_Categories['Country'] = reset_index_author_tot_only['Country'].copy()
Drug_Laws_Leniency_Scoring_Author_Categories['CategoryValue'] = Drug_Laws_Leniency_Scoring_Author_TotOnly['CategoryValue'].values.copy()
Drug_Laws_Leniency_Scoring_Author_Categories = Drug_Laws_Leniency_Scoring_Author_Categories.set_index('Country')
Drug_Laws_Leniency_Scoring_Author_Categories.to_csv('Drug_Laws_Leniency_Scoring_Author_Categories.csv')

Drug_Laws_Leniency_Scoring_Mean_Categories = pd.DataFrame()
Drug_Laws_Leniency_Scoring_Mean_Categories['Country'] = reset_index_mean_tot_only['Country'].copy()
Drug_Laws_Leniency_Scoring_Mean_Categories['CategoryValue'] = Drug_Laws_Leniency_Scoring_Mean_TotOnly['CategoryValue'].values.copy()
Drug_Laws_Leniency_Scoring_Mean_Categories = Drug_Laws_Leniency_Scoring_Mean_Categories.set_index('Country')
Drug_Laws_Leniency_Scoring_Mean_Categories.to_csv('Drug_Laws_Leniency_Scoring_Mean_Categories.csv')
Drug_Laws_Leniency_Scoring_Mean_Categories.head()


,CategoryValue
Country,
Austria,1
Belgium,1
Bulgaria,2
Croatia,2
Cyprus,3


## Checking for NANs for Visualisation and Analysis

In [20]:
# We performed this step as excel doesn't display NaNs in its cell, so we did this for easy table visualisation to know what was missing and how patchy individual country data was. Quick at a glance. 
with pd.ExcelWriter('CheckingForNAN.xlsx') as writer:  
    overdoseDeaths_Total_Europe_Clean.to_excel(writer, sheet_name='OverdoseDeathsTotal')
    n_UseOffences_Europe_Clean.to_excel(writer, sheet_name='nUseOffences')
    n_Offenders_Europe_clean.to_excel(writer, sheet_name='nOffenders')